In [1]:
import sqlite3


def record_customer_details():
    name = input("Enter your name: ")
    return name


class PizzaOrderingSystem:
    def __init__(self):
        self.menu = {
            "Margherita": 8,
            "Pepperoni": 10,
            "Vegetarian": 9
        }
        self.discount_codes = {
            "SAVE10": 10,
            "DEAL15": 15
        }
        # the code connects to sqlite database named pizza_biz.db
        # it makes it easy for application user to retrieve data later
        self.connection = sqlite3.connect('pizza_biz.db')
        self.cursor = self.connection.cursor()
        self.create_tables()

    # create orders table for database
    def create_tables(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS orders (
                                id INTEGER PRIMARY KEY,
                                name TEXT,
                                items TEXT,
                                total_cost REAL,
                                delivery_option TEXT,
                                delivery_address TEXT,
                                payment_method TEXT,
                                card_number TEXT,
                                expiry_date TEXT,
                                cvv TEXT
                            )''')
        self.connection.commit()

    def display_menu(self):
        print("Menu:")
        for item, price in self.menu.items():
            print(f"{item}: ${price}")

    def place_order(self):
        order = []
        total_cost = 0

        print("Place your order (type 'done' when finished):")
        while True:
            item = input("Enter item name: ")
            if item.lower() == 'done':
                break
            if item not in self.menu:
                print("Invalid item! Please choose from the menu.")
                continue
            order.append(item)
            total_cost += self.menu[item]

        print("Order Summary:")
        for item in order:
            print(f"{item}: ${self.menu[item]}")
        print(f"Total Cost: ${total_cost}")

        return order, total_cost

    def apply_discount(self, total_cost):
        discount_code = input("Enter discount code (if any): ")
        if discount_code in self.discount_codes:
            discount = self.discount_codes[discount_code]
            total_cost -= discount
            print(f"Discount applied: ${discount}")
        else:
            print("Invalid discount code!")

        return total_cost

    def select_delivery_option(self):
        delivery_option = input("Select delivery option (pickup/delivery): ")
        if delivery_option.lower() == 'delivery':
            delivery_address = input("Enter delivery address: ")
            return delivery_option, delivery_address
        elif delivery_option.lower() == 'pickup':
            return delivery_option, None
        else:
            print("Invalid option! Please select pickup or delivery.")
            return self.select_delivery_option()

    def select_payment_method(self):
        payment_method = input("Select payment method (cash/card): ")
        if payment_method.lower() == 'cash':
            return payment_method, None
        elif payment_method.lower() == 'card':
            card_number = input("Enter card number: ")
            expiry_date = input("Enter expiry date: ")
            cvv = input("Enter CVV: ")
            return payment_method, card_number, expiry_date, cvv
        else:
            print("Invalid payment method!")
            return self.select_payment_method()

    def record_payment_details(self):
        selected_method = self.select_payment_method()
        if selected_method[0] == 'card':
            payment_method, card_number, expiry_date, cvv = selected_method
        else:
            payment_method = selected_method
            card_number, expiry_date, cvv = None, None, None  # Set card details to None for cash payment
        return payment_method, card_number, expiry_date, cvv

    def store_customer_details(self, name, order, total_cost, delivery_option, delivery_address, payment_method,
                               card_number, expiry_date, cvv):
        # convert payment method from tuple to string if payment method is cash
        payment_method_to_str = payment_method[0] if isinstance(payment_method, tuple) else payment_method
        if delivery_address is None:
            delivery_address = ''  # Assign an empty string if delivery_address is None
        self.cursor.execute('''INSERT INTO orders 
                              (name, items, total_cost, delivery_option, delivery_address, payment_method, card_number, 
                              expiry_date, cvv) 
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                            (name, ', '.join(order), total_cost, delivery_option, delivery_address,
                             payment_method_to_str,
                             card_number, expiry_date, cvv))
        self.connection.commit()
        print("Customer Details Recorded Successfully!")
        print("Thank you for ordering with us.")

    def check_previous_deliveries(self):
        choice = input("Do you want to know previous deliveries? (yes/no): ")
        if choice.lower() == 'yes':
            self.display_previous_deliveries()
        elif choice.lower() == 'no':
            pass
        else:
            print("Invalid choice! Please enter 'yes' or 'no'.")
            return self.check_previous_deliveries()

    def display_previous_deliveries(self):
        self.cursor.execute("SELECT * FROM orders")
        orders = self.cursor.fetchall()
        if orders:
            print("Previous Deliveries:")
            for order in orders:
                print(f"Order ID: {order[0]}, Name: {order[1]}, Items: {order[2]}, Total Cost: ${order[3]}")
        else:
            print("No previous deliveries found.")

    def place_order_flow(self):
        self.display_menu()
        self.check_previous_deliveries()  # Ask user if they want to know previous deliveries
        order, total_cost = self.place_order()
        total_cost = self.apply_discount(total_cost)
        delivery_option, delivery_address = self.select_delivery_option()
        name = record_customer_details()
        payment_method, card_number, expiry_date, cvv = self.record_payment_details()
        self.store_customer_details(name, order, total_cost, delivery_option, delivery_address, payment_method,
                                    card_number, expiry_date, cvv)


# Main program
if __name__ == "__main__":
    pizza_ordering_system = PizzaOrderingSystem()
    pizza_ordering_system.place_order_flow()


Menu:
Margherita: $8
Pepperoni: $10
Vegetarian: $9


Do you want to know previous deliveries? (yes/no):  no


Place your order (type 'done' when finished):


Enter item name:  Margherita
Enter item name:  done


Order Summary:
Margherita: $8
Total Cost: $8


Enter discount code (if any):  SAVE10


Discount applied: $10


Select delivery option (pickup/delivery):  pickup
Enter your name:  jane smart
Select payment method (cash/card):  cash


Customer Details Recorded Successfully!
Thank you for ordering with us.
